In [2]:
import TradeClient
import asyncio
import nest_asyncio
import Analysis_new
import MarketDataFetcher
import DataStoreage
from typing import List, Union, Final, Optional
nest_asyncio.apply()

ins_market = MarketDataFetcher.FuturesMarket()

symbols = ['BTCUSDT', 'TRXUSDT', 'XRPUSDT', 'ADAUSDT', 'ETHUSDT']
data = {}
for symbol in symbols:
    data[symbol] = DataStoreage.KlineData()
    kline_data = asyncio.run(ins_market.fetch_klines_limit(symbol=symbol, interval='3m', limit=100))
    data[symbol].set_data(kline_data)

ins_analysis = Analysis_new.AnalysisManager(data)
ins_analysis.get_success_signal()

{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0]}}
{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0]}}
{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0]}}
{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0]}}
{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}}
{1: {'BTCUSDT': [0, 1, 1, 0]}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}}
{1: {'BTCUSDT': [0, 1, 1, 0], 'TRXUSDT': [0, 1, 1, 0]}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}}
{1: {'BTCUSDT': [0, 1, 1, 0], 'TRXUSDT': [0, 1, 1, 0], 'XRPUSDT': [0, 1, 1, 0]}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}}
{1

In [ ]:
trade_log.update_trade_data(timestamp=int(time.time()*1000)-(60000*3*20), price=200)
pprint(trade_log)

In [ ]:
# 전체 자산 관리와 거래 기록 관리

"""수동태로 동작하도록."""
class WalletManager:
    """
    전체 자산 관리와 거래기록을 관리한다.
    해당 데이터가 필요한 클라스들은 instance자체를 공유하기 때문에
    본 클라스에 대한 의존도가 높다. 본 클라스의 계산역할이 매우 중요함.
    속성명을 수정하지 말것.
    """

    def __init__(
        self,
        is_profit_preservation: bool = True,
        initial_balance: float = 1_000,
    ):
        self.trade_history: List[TradingLog] = []
        self.closed_positions: Dict[str, List[List[Any]]] = {}
        self.open_positions: Dict[str, List[List[Any]]] = {}
        self.number_of_stocks: int = 0
        self.initial_balance: float = initial_balance  # 초기 자산
        self.total_wallet_balance: float = initial_balance  # 총 평가 자산
        self.active_value: float = 0  # 거래 중 자산 가치
        self.available_balance: float = initial_balance  # 사용 가능한 예수금
        self.profit_loss: float = 0  # 손익 금액
        self.profit_loss_ratio: float = 0  # 손익률
        self.trade_count: int = 0  # 총 체결 횟수
        ### 정확한 공식을 대입하지 못해서 LiveTrading거래 종료시 강제 데이터 기입 로직 있음.

        ##=---=####=---=####=---=####=---=####=---=####=---=##
        # -=###=----=#=- DEBUG CODE           -=##=----=###=-#
        ##=---=####=---=####=---=####=---=####=---=####=---=##
        self.secured_profit: float = 0  # init_balance를 초과하는 수익금액
        self.is_profit_preservation: bool = is_profit_preservation
        # self.trading_log_attr_maps:Dict[str, int] = utils._info_trade_log_attr_maps()

    # 현재 TradingLog정보를 List[Dict] 타입으로 변환 후 pickle 형태로 저장한다.
    # def export_trading_logs(self, is_save: bool = True):
    #     file_name = "trade_history.pkl"
    #     # TradingLog의 속성 이름을 확보한다.
    #     keys_ = list(TradingLog.__annotations__)
    #     # 데이터를 저장할 빈 리스트를 생성한다.
    #     result = []
    #     for trade_log in self.trade_history:
    #         data_ = trade_log.to_list()
    #         temp_log = {}
    #         for idx, key in enumerate(keys_):
    #             temp_log[key] = data_[idx]
    #         result.append(temp_log)
    #     if is_save:
    #         with open(file_name, "wb") as file:
    #             pickle.dump(result, file)
    #         return result
    #     return result

    # 현재 진행중인 거래가 있는지 여부를 점검한다.
    def validate_open_position(self, symbol: str):
        convert_to_symbol = f"{self.market}_{symbol}"
        # open_position에 해당 symbol이 있을경우
        if convert_to_symbol in self.open_positions:
            return True
        # 없을경우
        else:
            return False

    # TradingLog 클라스를 컨테이너 데이터에 저장한다.
    def add_log_data(self, log_data: TradingLog):
        # symbol정보를 조회한다.
        # log값에 이미 market 데이터가 들어있다.
        symbol = log_data.symbol

        # container name은 symbol로 정하고 데이터는 TradingLog를 넣는다.
        self.data_container.set_data(data_name=symbol, data=log_data)
        # log data를 LIST형태로 반환한다.
        trade_data = self.__extract_valid_data(data=log_data)
        # 총 거래 횟수를 계산한다.
        self.trade_count += 1
        # list형태로 반환된 데이터를 open_position에 저장한다.
        self.open_positions[symbol] = trade_data
        self.update_data()

    # 데이터를 업데이트하는 동시에 stop 신호를 반환받는다.
    def update_log_data(self, symbol: str, price: float, timestamp: int) -> bool:
        convert_to_symbol = f"{self.market}_{symbol}"

        # container 데이터에 해당 symbol값이 없으면
        if not convert_to_symbol in self.data_container.get_all_data_names():
            # stop 거절 신호를 반환한다. (아무일도 발생하지 않음.)
            return False
        # 데이터를 업데이트 한다.
        self.data_container.get_data(data_name=convert_to_symbol).update_trade_data(
            current_price=price, current_timestamp=timestamp
        )

        # 컨테이너에서 데이터를 불러온다.
        log_data = self.data_container.get_data(data_name=convert_to_symbol)
        trade_data = self.__extract_valid_data(data=log_data)
        # print(trade_data)
        self.open_positions[convert_to_symbol] = trade_data
        # portpolid의 각 값들을 계산하여 업데이트 한다.
        self.update_data()
        return self.data_container.get_data(data_name=convert_to_symbol).stop_signal

    # 필요한 값만 추출하여 리스트 형태로 반환한다.
    def __extract_valid_data(self, data: TradingLog):
        return [
            data.start_timestamp,  # 0 시작 타임스템프
            data.last_timestamp,  # 1  종료 타임스템프
            data.position,  # 2    포지션 (1:long, 2:short)
            data.leverage,  # 3    레버리지
            data.quantity,  # 4    수량
            data.entry_price,  # 5 진입가격
            data.high_price,  # 6  최고가격
            data.low_price,  # 7   최저가격
            data.current_price,  # 8   현재가격 또는 마지막 가격
            data.stop_price,  # 9  stoploss 가격
            data.initial_value,  # 10  초기 진입 평가 금액
            data.current_value,  # 11  현재 평가 금액
            data.net_profit_loss,  # 12    수수료 제외 PnL
            data.gross_profit_loss,  # 13  수수료 포함 PnL
            data.entry_fee,  # 14  진입 수수료
            data.exit_fee,  # 15   종료 수수료
            data.trade_scenario,  # 16  시나리오 종류
        ]

    # 거래종료시 open_position 데이터를 정리한다.
    def remove_order_data(self, symbol: str):

        convert_to_symbol = f"{self.market}_{symbol}"
        # 해당 symbol이 거래중이 아니면 오류를 발생시킨다. 오류가 맞음.
        if not self.open_positions.get(convert_to_symbol):
            raise ValueError(f"진행중인 거래 없음: {symbol}")

        # open_positions 데이터를 복사 및 변수로 저장한다.
        open_position_data = self.open_positions[convert_to_symbol].copy()

        # open_position 데이터를 삭제한다.
        del self.open_positions[convert_to_symbol]

        # 거래종료 positions에 해당 symbol이 있으면,
        if convert_to_symbol in self.closed_positions.keys():
            # 데이터를 끝에 추가한다.
            self.closed_positions[convert_to_symbol].append(open_position_data)
        # 만약에 없으면
        elif not convert_to_symbol in self.closed_positions.keys():
            # 신규로 생성하여 데이터를 저장한다.
            self.closed_positions[convert_to_symbol] = [open_position_data]

        trade_log_data = self.data_container.get_data(data_name=convert_to_symbol)

        # 트레이드 히스토리에 현재 트레이드 로그값을 저장한다.
        self.trade_history.append(trade_log_data.__dict__)
        # 저장이 완료되면 트레이드 로그값은 삭제한다.

        # TEST 모드일경우 거래내역 출력
        # if not trade_log_data.test_mode:
        #     pprint(trade_log_data)

        self.data_container.remove_data(data_name=convert_to_symbol)
        # 각종 값들을 계산 및 업데이트 한다.
        self.update_data()

    # class의 속성값들을 계산 및 업데이트한다.
    def update_data(self):
        # 종료 거래값을 초기화
        closed_trade_data = []
        # 진행 거래값을 초기화
        open_trade_data = []

        # 종료 거래내역을 list타입으로 저장하여 closed_trade_data에 추가한다.
        for _, closed_data in self.closed_positions.items():
            for nest_closed_data in closed_data:
                closed_trade_data.append(nest_closed_data)
        # 진행 거래내역을 list타입으로 저장하여 open_trade_data에 추가한다.
        for _, open_data in self.open_positions.items():
            open_trade_data.append(open_data)

        # closed_trade_data를 np.ndarray 형태로 전환한다.
        closed_data_array = np.array(object=closed_trade_data, dtype=float)
        # closed_trade_data의 차원 배열수가 1개면
        if closed_data_array.ndim == 1:
            # 배열수의 형태를 변경한다.
            closed_data_array = closed_data_array.reshape(1, -1)

        # open_trade_data를 np.ndarray 형태로 전환한다.
        open_data_array = np.array(object=open_trade_data, dtype=float)
        # open_trade_data의 차원 배열수가 1개면
        if open_data_array.ndim == 1:
            # 배열수의 형태를 변환한다.
            open_data_array = open_data_array.reshape(1, -1)

        # open_data_array.size 예) [0,0,0,0,0,0,0]경우
        # 필요한 기능인가??
        if open_data_array.size == 0:
            # 현재 포지션 수익금
            open_pnl = 0
            # 진행중인 비용
            active_value = 0
        else:
            open_pnl = np.sum(open_data_array[:, 13])
            active_value = np.sum(open_data_array[:, 10])

        # closed_data_array.size 예) [0,0,0,0,0,0,0]경우
        # 필요한 기능인가?
        if closed_data_array.size == 0:
            # 종료 포지션 수익금
            closed_pnl = 0
        else:
            # print(closed_data_array)
            closed_pnl = np.sum(closed_data_array[:, 13])

        # open_pnl = np.sum(open_data_array[:, 2])

        # 현재 거래중인 포지션 수
        self.number_of_stocks = len(self.open_positions.keys())
        # 거래중인 금액
        self.active_value = active_value

        # 예수금
        self.available_balance = self.initial_balance + closed_pnl - self.active_value
        # 손익금
        self.profit_loss = closed_pnl + open_pnl
        # 총 평가 금액
        self.total_wallet_balance = self.profit_loss + self.initial_balance
        # 손익비율
        self.profit_loss_ratio = (
            self.total_wallet_balance - self.initial_balance
        ) / self.initial_balance

        ##=---=####=---=####=---=####=---=####=---=####=---=##
        # -=###=----=#=- DEBUG CODE           -=##=----=###=-#
        ##=---=####=---=####=---=####=---=####=---=####=---=##

        # 추가개발 계획
        # live_trad시 new_profit_to_secure 이체시키는 기능이 필요함.
        new_profit_to_secure = (
            self.total_wallet_balance - self.initial_balance - self.secured_profit
        )
        # 수익이 발생하고, 현재 거래중인 항목이 없을경우
        if (
            self.is_profit_preservation
            and new_profit_to_secure > 0
            and open_data_array.size == 0
        ):
            self.secured_profit += new_profit_to_secure

    # 원금을 초과하는 수익금액을 반환 및 수익금액 내역을 초기화한다.
    # 라이브 트레이딩시 해당 금액을 이체하기 위함이다.
    def get_secured_profit(self):
        """
        1. 기능 : 원금을 초과하는 수익금액을 반환 및 초기화 한다. 라이브 트레이딩 대응용
        2. 매개변수 : 해당없음.

        논리적 개선이 필요하다.

        """
        result = self.secured_profit.copy()
        self.secured_profit = 0
        return result

    # 계좌 수익금 이체 후 기초자금을 리셋한다.
    # 이체 후 해당 자업 미실행시 실제 계좌와 데이터 불균형 발생함.
    def reset_initial_balance(self, balance):
        """
        1. 기능 : 초기자금을 지정한 금액(balance)로 초기화 한다.
        2. 매개변수
            1) balance : 초기화할 금액
        3. 추가정보
            - 자금 이체 후 해당작업 미실행시 실제 계좌와 PortfolioManager값 데이터 불균형 발생함.

        """
        self.initial_balance = balance

In [5]:
from typing import List, Dict, Optional, Any, Union
from DataStoreage import *
from ConfigSetting import SystemConfig, InitialSetup, SymbolConfig
import os
import utils
import asyncio
import nest_asyncio
# import TradeClient
import importlib
import ConfigSetting
from DataStoreage import TradingLog
import time
import numpy as np

nest_asyncio.apply()



class WalletManager:
    def __init__(self, initial_balance:float = 1_000):
        self.trade_history: List[Dict[str, Any]] = []
        self.number_of_stocks: int = 0
        self.initial_balance: float = initial_balance  # 초기 자산
        self.total_wallet_balance: float = 0  # 총 평가 자산
        self.active_value: float = 0  # 거래 중 자산 가치
        self.available_balance: float = 0  # 사용 가능한 예수금
        self.profit_loss: float = 0  # 손익 금액
        self.profit_loss_ratio: float = 0  # 손익률
        self.trade_count: int = 0  # 총 체결 횟수

        ### system setting
        self.__start_timestamp: int = int(time.time() * 1_000)
        self.__log_data: List[TradingLog] = {1:{}, 2:{}}

    async def init_setting(self):
        InitialSetup.initialize()
        # testmode는 확인 후 파일 삭제할 것.
        file_name = {True:SystemConfig.test_trade_history,
                     False:SystemConfig.live_trade_history}
        
        path = os.path.join(SystemConfig.parent_folder_path,
                            SystemConfig.data_folder_name,
                            file_name[InitialSetup.mode])

        if os.path.isfile(path):
            trade_history = utils._load_json(path)
            for trade_log in trade_history:
                log_data = TradingLog(**trade_log)
                self.trade_history.append(log_data)

        if InitialSetup.mode:
            balance = self.initial_balance
        elif not InitialSetup.mode:
            ins_client = utils._get_import('TradeClient', f'{SystemConfig.market_type}Client')
            balance = await ins_client.get_total_wallet_balance()
        self.total_wallet_balance = self.available_balance = balance

    def add_log_data(self, log_data:TradingLog):
        symbol = TradingLog.symbol
        position = TradingLog.position
        self.__log_data[position][symbol] = TradingLog

    def update_log_data(self, )

    def update_wallet(self):
        if self.trade_history:
            self.profit_loss = np.array([log.gross_pnl for log in self.trade_history])
            self.active_value 
        else:
            self.active_value = 0
            self.profit_loss = 0

        self.profit_loss_ratio = self.profit_loss / self.initial_balance
        self.total_wallet_balance = self.initial_balance + self.profit_loss
        
                
        
            
            

In [3]:
import asyncio
import nest_asyncio
from WalletHandler import WalletManager

nest_asyncio.apply()
obj = WalletManager()
asyncio.run(obj.init_setting())


Mode set to: Test Mode Enabled (True)


In [7]:
import ConfigSetting

print(ConfigSetting.InitialSetup.mode)

None


In [5]:
text = "BTCUSDT"
binary = ''.join(format(ord(char), '08b') for char in text)

print(binary)  # 01000001 01000010 01000011


01000010010101000100001101010101010100110100010001010100


In [17]:
def text_to_decimal_str(text: str) -> str:
    """ 문자열을 10진수 문자열로 변환 (앞의 0 유지) """
    return ''.join(f"{ord(char):03d}" for char in text)  # 3자리 고정 (문자열 형태)

def decimal_str_to_text(decimal_str: str) -> str:
    """ 10진수 문자열을 원래 문자열로 복원 """
    return ''.join(chr(int(decimal_str[i:i+3])) for i in range(0, len(decimal_str), 3))

# 변환 실행
text = "1231USDT"
decimal_str = text_to_decimal_str(text)
print(f"10진수 문자열: {decimal_str}")  # 앞의 0이 유지됨

# 복원 실행
original_text = decimal_str_to_text(decimal_str)
print(f"복원된 문자열: {original_text}")


10진수 문자열: 049050051049085083068084
복원된 문자열: 1231USDT


In [14]:
def decimal_to_text(decimal: int) -> str:
    """ 10진수를 원래 문자열로 복원 (3자리 고정) """
    decimal_str = str(decimal)  # 정수를 문자열로 변환
    return ''.join(chr(int(decimal_str[i:i+3])) for i in range(0, len(decimal_str), 3))

# 복원 실행
original_text = decimal_to_text(066084067085083068084)
print(original_text)  # "BTCUSDT"


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2678265750.py, line 7)